# Домашнее задание 1

На семинаре мы пробовали извлекать сущность "Приставка". В домашнем задании попробуйте извлекать сущность "Игра для приставки" или "Аксессуар для приставки" (1 на выбор).  
Данные - https://github.com/mannefedov/compling_nlp_hse_course/blob/master/data/pristavki.csv

Напишите правила в yargy для извлечения одной из выбранных сущностей. Начните с самого простого правила и понемногу добавляйте условий. Остановитесь, когда на датасете в ~9к текстов у вас получится извлекать сущности хотя бы в 500 текстах (не обязательно уникальных сущностей)

https://yargy.readthedocs.io/ru/latest/index.html

In [1]:
from yargy import Parser, rule, or_
from yargy.predicates import custom
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact

import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [2]:
data = pd.read_csv('pristavki.csv', header=None, names=['text'])

In [11]:
Game = fact('Game', ['pre_title', 'title', 'sequel', 'title_append', 'etc'])

PRE_TITLE = ['Super', 'Ultimate', 'Super']

ETC = ['goty', 'game of the year', 'игра года', 'издание игра года', 'goty edition', 'remastered',
       'HD', 'Director\'s cut', 'Gold', 'Platinum', 'Advance', 'Trilogy', 'Gold Edition', 'Complete',
       'Arcade Edition', 'HD Remaster', 'HD Edtiton', 'расширенная версия']

SEQUEL = ['2', '3', '4', '5', '6', '7', '8' '9', '10', 'II', 'III', 'IV', 'V', 'VI', 'VII',
          'VIII', 'IX', 'X', 'XL', 'XI', 'XII', 'XIII', 'XIV', 'XV']

Witcher = rule(morph_pipeline(['The Witcher', 'Witcher', 'Ведьмак']).interpretation(Game.title),
               morph_pipeline(SEQUEL).interpretation(Game.sequel).optional(),
               morph_pipeline(['Wild Hunt', 'Дикая Охота', 'Assassins of Kings', 'Убийцы Королей']).
               interpretation(Game.title_append).optional(),
               morph_pipeline(ETC).interpretation(Game.etc).optional())

MK = rule(morph_pipeline(PRE_TITLE).interpretation(Game.pre_title).optional(),
          morph_pipeline(['Mortal Kombat', 'MK']).interpretation(Game.title),
          morph_pipeline(SEQUEL).interpretation(Game.sequel).optional(),
          morph_pipeline(['Deadly Alliance', 'Deception', 'Unchained', 'Armaggedon', 'vs. DC Universe']).
          interpretation(Game.title_append).optional(),
          morph_pipeline(ETC).interpretation(Game.etc).optional())

SF = rule(morph_pipeline(PRE_TITLE).interpretation(Game.pre_title).optional(),
         morph_pipeline(['Street Fighter', 'SF']).interpretation(Game.title),
         morph_pipeline(SEQUEL).interpretation(Game.sequel).optional(),
         morph_pipeline(ETC).interpretation(Game.etc).optional())

RE = rule(morph_pipeline(['Resident Evil', 'RE']).interpretation(Game.title),
          morph_pipeline(SEQUEL).interpretation(Game.sequel).optional(),
          morph_pipeline(['Code Veronica', 'Operation Racoon City', 'Umbrella Chronicles', 'Darkside Chronicles',
                         'Biohazard']).
          interpretation(Game.title_append).optional(),
          morph_pipeline(ETC).interpretation(Game.etc).optional())

AC = rule(morph_pipeline(['AC', 'Assassins creed', 'Assassin\'s Creed',
                          'asassins creed', 'assasins creed',
                          'ассасин крид', 'ассасинс крид',
                          'асассин крид', 'асассинс крид',
                          'ассассин крид', 'ассассинс крид',
                          'ассассин', 'асассин', 'ассассин']).interpretation(Game.title),
          morph_pipeline(SEQUEL).interpretation(Game.sequel).optional(),
          morph_pipeline(['Black Flag', 'Syndicate', 'Revelations', 'Rogue', 'Brotherhood',
                          'Unity', 'Черный Флаг', 'Синдикат', 'Откровения', 'Изгой', 'Еиднство']).
          interpretation(Game.title_append).optional(),
          morph_pipeline(ETC).interpretation(Game.etc).optional())

GAME = or_(Witcher, MK, SF, RE, AC).interpretation(Game)

parser = Parser(GAME)

In [12]:
matches = []

for sent in data.text:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [13]:
len(matches)

686

In [23]:
matches[:5]

[Game(pre_title=None,
      title='асассин',
      sequel='4',
      title_append=None,
      etc=None), Game(pre_title=None,
      title='Assassins Creed',
      sequel='4',
      title_append='Черный флаг',
      etc=None), Game(pre_title='Super',
      title='Street Fighter',
      sequel='5',
      title_append=None,
      etc=None), Game(pre_title=None,
      title='Ведьмак',
      sequel=None,
      title_append=None,
      etc=None), Game(pre_title=None,
      title='Assassins creed',
      sequel=None,
      title_append='unity',
      etc=None)]

In [29]:
[match for match in matches if match.pre_title]

[Game(pre_title='Super',
      title='Street Fighter',
      sequel='5',
      title_append=None,
      etc=None), Game(pre_title='Super',
      title='Street Fighter',
      sequel=None,
      title_append=None,
      etc=None)]